In [ ]:
import os
from google.colab import drive

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Unintentional_Unalignment/unintentional-unalignment') #cloned repo to drive

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers
!pip install -r requirements.txt

In [ ]:
from huggingface_hub import login

login(token='token')

In [ ]:
import wandb

wandb.login(key="token")

In [ ]:
!wget -O "data_files/persona/ends-justify-means.jsonl" "https://huggingface.co/datasets/Anthropic/model-written-evals/resolve/main/persona/ends-justify-means.jsonl"

Experiment 1: Section 3

SFT Finetuning

In [ ]:
!python persona_single_example_experiment_plan_runner.py --plan_config_path persona_experiments/experiments_plans/persona_sft_olmo1b_experiments_plan.json

In [ ]:
!python persona_single_example_experiment_plan_runner.py --plan_config_path persona_experiments/experiments_plans/persona_sft_gemma2b_experiments_plan.json

Experiments on models after SFT

In [ ]:
!python persona_single_example_experiment_plan_runner.py --plan_config_path persona_experiments/experiments_plans/persona_post_sft_olmo1b_experiments_plan.json

In [ ]:
!python persona_single_example_experiment_plan_runner.py --plan_config_path persona_experiments/experiments_plans/persona_post_sft_gemma2b_experiments_plan.json

Experiments on base models

In [ ]:
!python persona_single_example_experiment_plan_runner.py --plan_config_path persona_experiments/experiments_plans/persona_base_olmo1b_experiments_plan.json

In [ ]:
!python persona_single_example_experiment_plan_runner.py --plan_config_path persona_experiments/experiments_plans/persona_base_gemma2b_experiments_plan.json

Experiments on models with IPO

In [ ]:
!python persona_single_example_experiment_plan_runner.py --plan_config_path persona_experiments/experiments_plans/persona_post_sft_ipo_olmo1b_experiments_plan.json

In [ ]:
!python persona_single_example_experiment_plan_runner.py --plan_config_path persona_experiments/experiments_plans/persona_post_sft_ipo_gemma2b_experiments_plan.json

Experiment 2: Section 5

DPO

OLMo-1b model + Alpaca Farm

Modify compute_preference_similarity_per_example.py to include url directly:

if dataset_name == "tatsu-lab/alpaca_farm":
        url = "https://huggingface.co/datasets/tatsu-lab/alpaca_farm/resolve/main/alpaca_human_preference.json"
        return datasets.load_dataset("json", data_files=url, split="train", cache_dir=cache_dir)

Include float explicitely:

torch.quantile(ches_scores.float(), q=0.25)

torch.quantile(ln_ches_scores.float(), q=0.25)

torch.quantile(last_hidden_embedding_inner_prods.float(), q=0.25)

In [ ]:
!python compute_preference_similarity_per_example.py \
  --model allenai/OLMo-1B-hf \
  --dataset tatsu-lab/alpaca_farm \
  --output_dir outputs/pref_similarity \
  --num_train_samples 5000 \
  --gpu_id 0

Variations of the following code for quantile 0.0, 0.25, 0.5, 0.75, 1

In [ ]:
!python similarity_measures_experiment_runner.py \
  --run_name alpaca_olmo_ches_0.0 \
  --model allenai/OLMo-1B-hf \
  --preference_similarity_path "outputs/pref_similarity/OLMo-1B-hf_alpaca_farm/results_samples.pt" \
  --use_samples_with_preference_similarity_quantile 0.0 \
  --pref_similarity_measure ches_scores \
  --preference_dataset_path tatsu-lab/alpaca_farm \
  --model_parallel_without_accelerate \
  --seed -1 \
  --wandb_project alpaca_dpo \
  --preference_num_samples 512 \
  --batch_size_per_gpu_proc 4 \
  --gradient_accumulation_steps 8 \
  --kl_coeff 0.1 \
  --learning_rate 1e-7 \
  --num_train_epochs 1 \
  --eval_every_epochs 1 \
  --save_every_epochs -1

Variations of the following code for quantile 0.0, 0.25, 0.5, 0.75, 1

In [ ]:
!python similarity_measures_experiment_runner.py \
  --run_name alpaca_olmo_edit_0.0 \
  --model allenai/OLMo-1B-hf \
  --preference_similarity_path "outputs/pref_similarity/OLMo-1B-hf_alpaca_farm/results_samples.pt" \
  --use_samples_with_preference_similarity_quantile 0.0 \
  --pref_similarity_measure minus_normalized_edit_distances \
  --preference_dataset_path tatsu-lab/alpaca_farm \
  --model_parallel_without_accelerate \
  --seed -1 \
  --wandb_project alpaca_dpo \
  --preference_num_samples 512 \
  --batch_size_per_gpu_proc 4 \
  --gradient_accumulation_steps 8 \
  --kl_coeff 0.1 \
  --learning_rate 1e-7 \
  --num_train_epochs 1 \
  --eval_every_epochs 1 \
  --save_every_epochs -1

Variations of the following code for quantile 0.0, 0.25, 0.5, 0.75, 1

In [ ]:
!python similarity_measures_experiment_runner.py \
  --run_name alpaca_olmo_hemm_0.0 \
  --model allenai/OLMo-1B-hf \
  --preference_similarity_path "outputs/pref_similarity/OLMo-1B-hf_alpaca_farm/results_samples.pt" \
  --use_samples_with_preference_similarity_quantile 0.0 \
  --pref_similarity_measure last_hidden_embedding_inner_prods \
  --preference_dataset_path tatsu-lab/alpaca_farm \
  --model_parallel_without_accelerate \
  --seed -1 \
  --wandb_project alpaca_dpo \
  --preference_num_samples 512 \
  --batch_size_per_gpu_proc 4 \
  --gradient_accumulation_steps 8 \
  --kl_coeff 0.1 \
  --learning_rate 1e-7 \
  --num_train_epochs 1 \
  --eval_every_epochs 1 \
  --save_every_epochs -1

Gemma-2-9b + AlpacaFarm

In [ ]:
!python compute_preference_similarity_per_example.py \
--model google/gemma-2-9b \
--dataset tatsu-lab/alpaca_farm \
--output_dir outputs/pref_similarity \
--num_train_samples 5000 \
--gpu_id 0

Variations of the following code for quantile 0.0, 0.25, 0.5, 0.75, 1

In [ ]:
!python similarity_measures_experiment_fix.py \
--run_name alpaca_gemma9b_ches_0.00\
--model google/gemma-2-9b \
--preference_similarity_path outputs/pref_similarity/gemma-2-9b_alpaca_farm/results_samples.pt\
--use_samples_with_preference_similarity_quantile 0.00 \
--pref_similarity_measure ches_scores \
--preference_dataset_path tatsu-lab/alpaca_farm \
--model_parallel_without_accelerate \
--seed -1 \
--wandb_project alpaca_dpo \
--preference_num_samples 1024 \
--batch_size_per_gpu_proc 4 \
--gradient_accumulation_steps 8 \
--kl_coeff 0.1 \
--learning_rate 1e-6 \
--num_train_epochs 3 \
--eval_every_epochs 1 \
--save_every_epochs -1

Variations of the following code for quantile 0.0, 0.25, 0.5, 0.75, 1

In [ ]:
!python similarity_measures_experiment_fix.py \
--run_name alpaca_gemma9b_edit_0.00\
--model google/gemma-2-9b \
--preference_similarity_path outputs/pref_similarity/gemma-2-9b_alpaca_farm/results_samples.pt\
--use_samples_with_preference_similarity_quantile 0.00 \
--pref_similarity_measure minus_normalized_edit_distances \
--preference_dataset_path tatsu-lab/alpaca_farm \
--model_parallel_without_accelerate \
--seed -1 \
--wandb_project alpaca_dpo \
--preference_num_samples 1024 \
--batch_size_per_gpu_proc 4 \
--gradient_accumulation_steps 8 \
--kl_coeff 0.1 \
--learning_rate 1e-6 \
--num_train_epochs 3 \
--eval_every_epochs 1 \
--save_every_epochs -1

Variations of the following code for quantile 0.0, 0.25, 0.5, 0.75, 1

In [ ]:
!python similarity_measures_experiment_fix.py \
--run_name alpaca_gemma9b_hemm_0.00\
--model google/gemma-2-9b \
--preference_similarity_path outputs/pref_similarity/gemma-2-9b_alpaca_farm/results_samples.pt\
--use_samples_with_preference_similarity_quantile 0.00 \
--pref_similarity_measure last_hidden_embedding_inner_prods \
--preference_dataset_path tatsu-lab/alpaca_farm \
--model_parallel_without_accelerate \
--seed -1 \
--wandb_project alpaca_dpo \
--preference_num_samples 1024 \
--batch_size_per_gpu_proc 4 \
--gradient_accumulation_steps 8 \
--kl_coeff 0.1 \
--learning_rate 1e-6 \
--num_train_epochs 3 \
--eval_every_epochs 1 \
--save_every_epochs -1

(Also ran for Gemma-2b + Ultrafeedback)

In [ ]:
python compute_preference_similarity_per_example.py \
--model google/gemma-2-9b \
--dataset HuggingFaceH4/ultrafeedback_binarized \
--output_dir outputs/pref_similarity \
--num_train_samples 5000 \
--train_samples_random_seed 548 \
--gpu_id 0

In [ ]:
!python similarity_measures_experiment_fix.py \
--run_name alpaca_gemma9b_ches_0.00\
--model google/gemma-2-9b \
--preference_similarity_path outputs/pref_similarity/gemma-2-9b_alpaca_farm/results_samples.pt\
--use_samples_with_preference_similarity_quantile 0.00 \
--pref_similarity_measure ches_scores \
--preference_dataset_path HuggingFaceH4/ultrafeedback_binarized \
--model_parallel_without_accelerate \
--seed -1 \
--wandb_project ultrafeedback_dpo \
--preference_num_samples 512 \
--batch_size_per_gpu_proc 4 \
--gradient_accumulation_steps 8 \
--kl_coeff 0.1 \
--learning_rate 1e-7 \
--num_train_epochs 1 \
--eval_every_epochs 1 \
--save_every_epochs -1

IPO

OLMo-1b + AlpacaFarm

Variations of the following code for quantile 0.0, 0.25, 0.5, 0.75, 1

In [ ]:
!python similarity_measures_experiment_runner.py \
  --run_name alpaca_olmo_ches_0.0 \
  --model allenai/OLMo-1B-hf \
  --preference_similarity_path "outputs/pref_similarity/OLMo-1B-hf_alpaca_farm/results_samples.pt" \
  --use_samples_with_preference_similarity_quantile 0.0 \
  --pref_similarity_measure ches_scores \
  --preference_dataset_path tatsu-lab/alpaca_farm \
  --objective ipo \
  --model_parallel_without_accelerate \
  --seed -1 \
  --wandb_project alpaca_ipo \
  --preference_num_samples 512 \
  --batch_size_per_gpu_proc 4 \
  --gradient_accumulation_steps 8 \
  --kl_coeff 0.1 \
  --learning_rate 1e-7 \
  --num_train_epochs 1 \
  --eval_every_epochs 1 \
  --save_every_epochs -1

Visualisation

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
from pathlib import Path

In [ ]:
root_path = Path("./outputs/alpaca_dpo/gemma") #run for other similar paths

def convert_all_jsonl_in_folder(root_dir):
    if not root_dir.exists():
        print(f"Error: Directory '{root_dir}' not found. Please check the path.")
        return

    for experiment_folder in root_dir.iterdir():

        if experiment_folder.is_dir():
            print(f"Processing folder: {experiment_folder.name}")

            jsonl_files = list(experiment_folder.glob("*.jsonl"))

            if not jsonl_files:
                print("  -> No .jsonl files found.")
                continue

            for jsonl_file in jsonl_files:
                try:
                    df = pd.read_json(jsonl_file, lines=True)

                    txt_file_path = jsonl_file.with_suffix('.txt')

                    df.to_string(txt_file_path, index=False)

                    print(f"  -> Converted: {jsonl_file.name}  ==>  {txt_file_path.name}")

                except ValueError:
                    print(f"  -> Skipping {jsonl_file.name} (File might be empty or corrupt)")
                except Exception as e:
                    print(f"  -> Error converting {jsonl_file.name}: {e}")

            print("-" * 40)

convert_all_jsonl_in_folder(root_path)

In [ ]:
experiments_config = [
    {"path": "./outputs/alpaca_dpo/olmo", "label": "OLMo-1b (DPO, AlpacaFarm)"},
    {"path": "./outputs/alpaca_ipo/olmo", "label": "OLMo-1b (IPO, AlpacaFarm)"},
]

def parse_experiments(root_dir_str, method_label):
    data = []
    root_dir = Path(root_dir_str)

    if not root_dir.exists():
        print(f"Warning: Directory '{root_dir}' not found. Skipping.")
        return pd.DataFrame()

    folder_pattern = re.compile(r"^(?P<dataset>[a-z0-9]+)_(?P<model>[a-z0-9]+)_(?P<type>[a-z]+)_.*quantile_(?P<quantile>[\d\.]+)_")

    print(f"Scanning {method_label} at {root_dir}...")

    for folder in root_dir.iterdir():
        if not folder.is_dir():
            continue

        match = folder_pattern.search(folder.name)
        if not match:
            continue

        sim_type = match.group("type")
        quantile = float(match.group("quantile")) * 100

        txt_path = folder / "eval_metrics.txt"
        jsonl_path = folder / "eval_metrics.jsonl"

        df = None
        try:
            if txt_path.exists():
                df = pd.read_csv(txt_path, sep=r'\s+')
            elif jsonl_path.exists():
                df = pd.read_json(jsonl_path, lines=True)
        except Exception:
            continue

        if df is None or df.empty:
            continue

        change_cols = [c for c in df.columns if "preferred_logprobs_change_mean" in c]
        logprob_cols = [c for c in df.columns if "preferred_logprobs_mean" in c and "reference" not in c and "change" not in c]

        final_val = 0.0

        if change_cols:
            col_name = change_cols[0]
            final_val = df[col_name].iloc[-1]
        elif logprob_cols:
            col_name = logprob_cols[0]
            final_val = df[col_name].iloc[-1] - df[col_name].iloc[0]

        data.append({
            "Method": method_label,
            "Similarity Type": sim_type,
            "Preference Similarity (Percentile)": quantile,
            "Change in Preferred Log Probability": final_val
        })

    return pd.DataFrame(data)

all_dataframes = []

for exp in experiments_config:
    df = parse_experiments(exp["path"], exp["label"])
    all_dataframes.append(df)

if all_dataframes:
    df_results = pd.concat(all_dataframes, ignore_index=True)
else:
    df_results = pd.DataFrame()

if not df_results.empty:
    print(f"\nSuccessfully extracted {len(df_results)} data points.")

    name_map = {
        "ches": "CHES Score",
        "edit": "Edit Distance Similarity",
        "hidden": "Hidden Embedding Similarity",
        "hemm": "Hidden Embedding Similarity"
    }
    df_results["Similarity Type"] = df_results["Similarity Type"].map(name_map).fillna(df_results["Similarity Type"])

    unique_methods = df_results["Method"].unique()

    for method in unique_methods:
        subset_data = df_results[df_results["Method"] == method]

        plt.figure(figsize=(6, 5))

        sns.lineplot(
            data=subset_data,
            x="Preference Similarity (Percentile)",
            y="Change in Preferred Log Probability",
            hue="Similarity Type",
            style="Similarity Type",
            markers=True,
            dashes=False,
            linewidth=2.5,
            palette=["#2ca02c", "#d62728", "#1f77b4"]
        )

        plt.title(method) #sets the title to the method name
        plt.axhspan(-3, 0, color='red', alpha=0.1, label='Likelihood Displacement')
        plt.axhline(0, color='black', linewidth=0.8, linestyle='--')
        plt.yticks([-5, 0, 5, 10, 15])
        plt.xticks([0, 25, 50, 75, 100])
        plt.grid(True, alpha=0.3)
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

        clean_filename = re.sub(r'[^a-zA-Z0-9]', '_', method) + ".png"

        plt.savefig(clean_filename, dpi=300, bbox_inches='tight')
        plt.show()
        print(f"Saved graph to: {clean_filename}")

else:
    print("No data found! Please check your paths in experiments_config.")

Experiment 3: Section 6

Gemma-2b-it + SORRY-Bench

Modify the sorrybench_create_preferences_dataset.py file

with gemma-2b,we get:
  
ValueError: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed.


In [ ]:
!python sorrybench_create_preferences_dataset.py \
--model google/gemma-2b-it \
--use_rm \
--gpu_id 0 \
--random_seed 128

In [ ]:
!python sorrybench_evaluate_refusal_rate.py \
--models google/gemma-2b-it \
--dataset_path data_files/sorrybench/gemma-2b-it_seed_128_temp_1_topp_1_test_0-15_use_pairrm_2025-12-18_00-20-23 \
--output_dir outputs/sorrybench_refusal_rates \
--gpu_id 0

In [ ]:
!python compute_preference_similarity_per_example.py \
--model google/gemma-2b-it \
--dataset data_files/sorrybench/gemma-2b-it_seed_128_temp_1_topp_1_test_0-15_use_pairrm_2025-12-18_00-20-23/train.jsonl \
--output_dir outputs/pref_similarity \
--custom_dataset_display_name sorrybench \
--max_input_length -1 \
--gpu_id 0

In [ ]:
!python sorrybench_experiment_runner.py \
--run_name gemma_dpo_sft \
--model google/gemma-2b-it \
--dataset_path data_files/sorrybench/gemma-2b-it_seed_128_temp_1_topp_1_test_0-15_use_pairrm_2025-12-18_00-20-23 \
--model_parallel_without_accelerate \
--wandb_project sorrybench_dpo \
--batch_size_per_gpu_proc 4 \
--gradient_accumulation_steps 8 \
--learning_rate 5e-6 \
--num_train_epochs 3 \
--eval_every_epochs 1 \
--save_every_epochs -1 \
--refusal_eval_batch_size 16 \
--sft_coeff 1

In [ ]:
!python sorrybench_experiment_runner.py \
--run_name gemma_dpo_filtered_ches \
--model google/gemma-2b \
--dataset_path data_files/sorrybench/gemma-2b-it_seed_128_temp_1_topp_1_test_0-15_use_pairrm_2025-12-18_00-20-23 \
--preference_similarity_path outputs/pref_similarity/gemma-2b-it_sorrybench/results_samples.pt \
--pref_similarity_measure ln_ches_scores \
--preference_num_samples 19 \
--model_parallel_without_accelerate \
--wandb_project sorrybench_dpo \
--batch_size_per_gpu_proc 4 \
--gradient_accumulation_steps 8 \
--learning_rate 5e-6 \
--num_train_epochs 3 \
--eval_every_epochs 1 \
--save_every_epochs -1 \
--refusal_eval_batch_size 16 \
--sft_coeff 0

In [ ]:
!python sorrybench_experiment_runner.py \
--run_name gemma_dpo_gold \
--model google/gemma-2b-it \
--dataset_path data_files/sorrybench/gemma-2b-it_seed_128_temp_1_topp_1_test_0-15_use_pairrm_2025-12-18_00-20-23 \
--use_gold_preferences \
--model_parallel_without_accelerate \
--wandb_project sorrybench_dpo \
--batch_size_per_gpu_proc 4 \
--gradient_accumulation_steps 8 \
--learning_rate 1e-6 \
--num_train_epochs 3 \
--eval_every_epochs 1 \
--save_every_epochs -1 \
--refusal_eval_batch_size 16 \
--sft_coeff 0

Visualisation

In [ ]:
outputs_dir = Path("./outputs/sorrybench_dpo")
model_name = "Gemma-2B-IT"

base_model_train_eval = 0.8081081081081081

colors = {
    "Initial": "#666666",
    "DPO": "#D9534F",
    "DPO + SFT": "#5CB85C",
    "DPO (gold)": "#F0AD4E",
    "DPO (filtered)": "#5BC0DE"
}

mapping = [
    ("gold", "DPO (gold)"),
    ("sft", "DPO + SFT"),
    ("filter", "DPO (filtered)"),
    ("dpo", "DPO"),
]

def load_refusal_data(root_dir):
    data = []

    data.append({
        "Method": "Initial",
        "Refusal Rate (%)": base_model_train_eval * 100,
        "Run": "Base"
    })

    files = list(root_dir.glob("**/*_refusal_eval.json")) + list(root_dir.glob("**/*_refusal_eval.txt"))

    for file_path in files:
        folder_name = file_path.parent.name.lower()
        method_label = "Unknown"

        for keyword, label in mapping:
            if keyword in folder_name:
                method_label = label
                break

        refusal_val = None

        with open(file_path, 'r') as f:
            content = json.load(f)
            refusal_val = content.get("train_refusal_rate")

        if refusal_val is not None:
            rate_percent = float(refusal_val) * 100
            data.append({
                "Method": method_label,
                "Refusal Rate (%)": rate_percent,
                "Run": file_path.parent.name
            })
    return pd.DataFrame(data)

def aggregate_stats(df):
    stats = df.groupby("Method")["Refusal Rate (%)"].agg(['mean', 'min', 'max']).reset_index()

    stats['err_lower'] = stats['mean'] - stats['min']
    stats['err_upper'] = stats['max'] - stats['mean']

    custom_order = ["Initial", "DPO", "DPO + SFT", "DPO (gold)", "DPO (filtered)"]
    stats['Method'] = pd.Categorical(stats['Method'], categories=custom_order, ordered=True)
    stats = stats.sort_values('Method').dropna()

    return stats

def plot_figure_3_style(stats_df):
    plt.figure(figsize=(9, 6))

    x_pos = np.arange(len(stats_df))
    methods = stats_df["Method"].tolist()
    means = stats_df["mean"].tolist()

    yerr = [stats_df['err_lower'].tolist(), stats_df['err_upper'].tolist()]

    bar_colors = [colors.get(m, "#333333") for m in methods]

    bars = plt.bar(x_pos, means,
                   yerr=yerr,
                   capsize=6,
                   color=bar_colors,
                   edgecolor='white',
                   width=0.7,)

    max_y_limit = 0
    for rect, mean_val, method in zip(bars, means, methods):
        height = rect.get_height()

        row = stats_df[stats_df['Method'] == method].iloc[0]
        top_point = row['max']

        plt.text(rect.get_x() + rect.get_width()/2.0,
                 top_point + 2,
                 f'{mean_val:.1f}%',
                 ha='center', va='bottom', fontsize=11, fontweight='bold', color='black')

        if top_point > max_y_limit:
            max_y_limit = top_point

    plt.ylabel("Refusal Rate (%)", fontsize=12, fontweight='bold')
    plt.title(f"{model_name} Training Refusal Rates", fontsize=14)
    plt.xticks(x_pos, methods, fontsize=10)

    plt.ylim(0, max_y_limit * 1.15)

    plt.grid(axis='y', linestyle='--', alpha=0.5)
    plt.gca().set_axisbelow(True)


    patches = [plt.Rectangle((0,0),1,1, color=colors[label]) for label in methods]
    plt.legend(patches, methods, loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=3, frameon=False)

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    if not outputs_dir.exists():
        print(f"NOTE: Directory {outputs_dir} not found.")
        print("Please edit the 'outputs_dir' variable at the top of the script.")
    else:
        df = load_refusal_data(outputs_dir)

        if not df.empty:
            stats_df = aggregate_stats(df)
            print("\nAggregated Stats Table:")
            print(stats_df[['Method', 'mean', 'min', 'max']].to_string(index=False))
            plot_figure_3_style(stats_df)
        else:
            print("No data found to plot. Check your folder structure.")